### AWS IAM Identity Center (SSO)
##### AWS Command Line Interface (AWS CLI) Version 2 Requirement
https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html

In [ ]:
import os

os.system('curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "../awscliv2.zip"')
os.system('unzip -o ../awscliv2.zip -d ../')
os.system('cd .. && sudo ./aws/install --update')
os.system('aws --version')

##### Python Library for Jupyter Notebooks
https://github.com/botoplus/botoplus

In [ ]:
import sys

!{sys.executable} -m pip install botoplus --upgrade

##### Aqueduct Utility for IAM Identity Center (SSO) Authenication
https://github.com/jblukach/aqueduct

In [ ]:
import sys

!{sys.executable} -m pip install aqueduct-utility --upgrade

##### Login Example

```
Identity Store: portal
SSO Region: us-east-2
SSO Role: AWSAdministratorAccess
CLI Region: us-east-2
CLI Output: json
Authenticated!!
```

In [ ]:
import aqueduct.identity as _idp

_idp.login()

##### Logout Example

```
Logged Out!!
```

In [ ]:
import aqueduct.identity as _idp

_idp.logout()

##### Local Development

In [ ]:
import os

os.system('python setup.py install --user')

##### Build Test

In [ ]:
import botoplus.botoplus as botoplus

botoplus.hello()

### Amazon Security Lake
##### Identify Delegated Administrator

In [ ]:
import botoplus.botoplus as botoplus

selected_account = 'Unavailable'

session = botoplus.default()
client = session.client('organizations')
paginator = client.get_paginator('list_delegated_administrators')
pages = paginator.paginate()

for page in pages:
    for item in page['DelegatedAdministrators']:

        paginator2 = client.get_paginator('list_delegated_services_for_account')
        pages2 = paginator2.paginate(
            AccountId = item['Id']
        )

        for page2 in pages2:
            for item2 in page2['DelegatedServices']:
                if item2['ServicePrincipal'] == 'securitylake.amazonaws.com':
                    selected_account = item['Name']

print('Amazon Security Lake Delegated Administrator: '+selected_account)

##### Regions Ingesting Data

In [ ]:
import botoplus.botoplus as botoplus

ingestion = []

try:

    session = botoplus.defaults(selected_account)
    securitylake = session.client('securitylake')
    regions = securitylake.get_data_lake_organization_configuration()

    for region in regions['autoEnableNewAccount']:
        print(region['region'])
        ingestion.append(region['region'])
        for source in region['sources']:
            print(' * '+source['sourceVersion']+' '+source['sourceName'])

except:
    print('Amazon Security Lake Delegated Administrator: Unidentified')
    pass

##### Identify Replication Region

In [ ]:
import botoplus.botoplus as botoplus

replication = []

try:

    session = botoplus.defaults(selected_account)
    securitylake = session.client('securitylake')
    lakes = securitylake.list_data_lakes(
        regions = ingestion
    )

    for lake in lakes['dataLakes']:
        try:
            for region in lake['replicationConfiguration']['regions']:
                replication.append(region)
        except:
            pass

except:
    print('Amazon Security Ingestion Region(s): Unidentified')
    pass

try:
    replication = list(set(replication))
    replication = replication[0]
    print('Amazon Security Lake Replication Region: '+str(replication))
except:
    pass

##### Lifecycle Configuration

In [ ]:
import json
import botoplus.botoplus as botoplus

try:

    session = botoplus.defaults(selected_account)
    securitylake = session.client('securitylake')
    lakes = securitylake.list_data_lakes(
        regions = ingestion
    )

    for lake in lakes['dataLakes']:
        if lake['region'] == replication:
            print(json.dumps(lake['lifecycleConfiguration'], indent=4, sort_keys=True))

except:
    print('Amazon Security Ingestion Region(s): Unidentified')
    pass

### Amazon Athena
##### List Query Executions

In [ ]:
import time
import botoplus.botoplus as botoplus

try:

    session = botoplus.defaults(selected_account)
    athena = session.client('athena', region_name=replication)
    paginator = athena.get_paginator('list_query_executions')
    pages = paginator.paginate()

    for page in pages:
        for item in page['QueryExecutionIds']:
            execution = athena.get_query_execution(
                QueryExecutionId = item
            )
            if execution['QueryExecution']['Status']['SubmissionDateTime'].timestamp() > (time.time() - 604800):
                if execution['QueryExecution']['Status']['State'] == 'SUCCEEDED':
                    if execution['QueryExecution']['Statistics']['DataScannedInBytes'] > 0:
                        print(execution['QueryExecution']['Query'])
                        print(' * Date: '+str(execution['QueryExecution']['Status']['CompletionDateTime']))
                        print(' * Execution: '+execution['QueryExecution']['QueryExecutionId'])
                        print(' * Megabytes: '+str(execution['QueryExecution']['Statistics']['DataScannedInBytes']/1000000))
                        print(' * Minutes: '+str(execution['QueryExecution']['Statistics']['EngineExecutionTimeInMillis']/60000))

except:
    print('Amazon Security Ingestion Region(s): Unidentified')
    pass

##### Query Execution Output

In [ ]:
import botoplus.botoplus as botoplus

query_execution_id = botoplus.execution()

try:

    session = botoplus.defaults(selected_account)
    athena = session.client('athena', region_name=replication)
    output = athena.get_query_runtime_statistics(
        QueryExecutionId = query_execution_id
    )
    print(' * Megabytes: '+str(output['QueryRuntimeStatistics']['OutputStage']['OutputBytes']/1000000))
    print(' * Output Rows: '+str(output['QueryRuntimeStatistics']['OutputStage']['OutputRows']))

except:
    print('Amazon Security Ingestion Region(s): Unidentified')
    pass

##### Query Execution Output

In [ ]:
import botoplus.botoplus as botoplus
import pandas as pd

session = botoplus.defaults(selected_account)
athena = session.client('athena', region_name=replication)
output = athena.get_query_execution(
    QueryExecutionId = query_execution_id
)

bucket = output['QueryExecution']['ResultConfiguration']['OutputLocation']
out = bucket[5:].split('/')
print(bucket)

s3 = session.resource('s3')
s3.Object(out[0], out[1]).download_file('/tmp/'+out[1])

df = pd.read_csv('/tmp/'+out[1], sep=',')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df.head(1)

### Amazon Route 53 Resolver Query Logs
##### Start Query Execution

In [ ]:
# Multi-line athena search
import botoplus.botoplus as botoplus

query = """
SELECT time,
	metadata.product.version,
	metadata.product.name AS service,
	metadata.product.feature.name,
	metadata.product.vendor_name,
	metadata.profiles,
	metadata.version AS securitylake,
	src_endpoint.vpc_uid AS src_vpc_uid,
	src_endpoint.ip AS src_ip,
	src_endpoint.port AS src_port,
	src_endpoint.instance_uid AS src_instance_uid,
	query.hostname AS query_hostname,
	query.type AS query_type,
	query.class AS query_class,
	connection_info.protocol_name,
	connection_info.direction,
	connection_info.direction_id,
	dst_endpoint.instance_uid AS dst_instance_uid,
	dst_endpoint.interface_uid AS dst_inaterface_uid,
	severity_id,
	severity,
	class_name,
	class_uid,
	disposition,
	disposition_id,
	rcode_id,
	rcode,
	activity_id,
	activity_name,
	type_name,
	type_uid,
	unmapped,
	region,
	accountid,
	eventday,
	cardinality(answers) AS answer_count,
	answers
FROM amazon_security_lake_glue_db_us_east_2.amazon_security_lake_table_us_east_2_route53_1_0
WHERE eventDay BETWEEN cast(
		date_format(current_timestamp - INTERVAL '7' day, '%Y%m%d%H') as varchar
	)
	and cast(
		date_format(current_timestamp - INTERVAL '0' day, '%Y%m%d%H') as varchar
	)
ORDER BY time DESC
"""

converted = botoplus.convert(selected_account)

session = botoplus.defaults(selected_account)
athena = session.client('athena', region_name=replication)
athena.start_query_execution(
    QueryString = query,
    ResultConfiguration = {
        'OutputLocation': 's3://temp-athena-results-'+converted['awsaccount']+'-'+replication+'/'
    }
)